In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uhack-sentiments-20/submission.csv
/kaggle/input/uhack-sentiments-20/train.csv
/kaggle/input/uhack-sentiments-20/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/uhack-sentiments-20/train.csv")
test = pd.read_csv("/kaggle/input/uhack-sentiments-20/test.csv")
samp = pd.read_csv("/kaggle/input/uhack-sentiments-20/submission.csv")

In [3]:
train['Review'] = train['Review'].str.strip().str.replace("  "," ")
test['Review'] = test['Review'].str.strip().str.replace("  "," ")

In [4]:
train.head(3)

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0,For some reason everybody complains and I'm co...,0,0,0,0,0,0,0,0,0,0,1,0
1,1,"I like everything about it, great choice of sp...",0,0,0,0,1,1,0,0,0,0,0,1
2,2,Excellent ceiling fan brace. Easy to install a...,0,0,0,0,0,0,1,0,0,1,0,1


In [5]:
!pip install simpletransformers
!pip uninstall -y tokenizers
!pip install transformers

     |████████████████████████████████| 247 kB 593 kB/s            
     |████████████████████████████████| 43 kB 1.5 MB/s             
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 9.2 MB 5.9 MB/s            
     |████████████████████████████████| 4.3 MB 54.1 MB/s            
     |████████████████████████████████| 76 kB 5.3 MB/s             
     |████████████████████████████████| 82 kB 1.0 MB/s             
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=9ebd3e4dec8353b425ae5c272814dde618e745229cfb10b93730828d0749619b
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: Click
    Found existing installation: click 8.0.3
    Uninstalling click-8.0.3:
      Successfully uninstalled click-8.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This 

In [6]:
def random_seed(seed_value):
    import random 
    random.seed(seed_value)  
    import numpy as np
    np.random.seed(seed_value)  
    import torch
    torch.manual_seed(seed_value)  
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True   
        torch.backends.cudnn.benchmark = False

In [7]:
import os
import gc
import re
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, log_loss

In [8]:
train[train.duplicated()]

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity


In [9]:
test[test.duplicated()]

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity


In [10]:
for target in train.columns[2:].tolist():
    print(train[target].value_counts().to_frame())
    print("****")

   Components
0        5862
1         274
****
   Delivery and Customer Support
0                           5957
1                            179
****
   Design and Aesthetics
0                   5486
1                    650
****
   Dimensions
0        5441
1         695
****
   Features
0      5823
1       313
****
   Functionality
0           3598
1           2538
****
   Installation
0          5374
1           762
****
   Material
0      5981
1       155
****
   Price
0   5282
1    854
****
   Quality
0     3959
1     2177
****
   Usability
0       4907
1       1229
****
   Polarity
1      4770
0      1366
****


In [11]:
print(train['Review'].apply(lambda x: len(x.split())).describe())

count    6136.000000
mean       30.008475
std        32.793149
min         2.000000
25%        10.000000
50%        19.000000
75%        38.000000
max       374.000000
Name: Review, dtype: float64


In [12]:
print(test['Review'].apply(lambda x: len(x.split())).describe([0.8,0.9,0.95,0.99]))

count    2631.000000
mean       30.384265
std        36.614156
min         2.000000
50%        19.000000
80%        45.000000
90%        68.000000
95%        89.500000
99%       154.000000
max       649.000000
Name: Review, dtype: float64


In [13]:
train['labels'] = train.iloc[:,2:].values.tolist()

In [14]:
train = train[['Review', 'labels']]
test = test[['Review']]

train = train.rename(columns={'Review' : 'text'})
test = test.rename(columns={'Review' : 'text'})

In [15]:
from simpletransformers.classification import ClassificationModel
from simpletransformers.classification import MultiLabelClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, log_loss
from scipy.special import softmax
import warnings
warnings.filterwarnings("ignore")

In [16]:
model_args = {'train_batch_size': 8, 
              'reprocess_input_data': True,
              'overwrite_output_dir': True,
              'fp16': False,
#               'do_lower_case': False,
              'num_train_epochs': 6, #4
              'max_seq_length': 224,
              'regression': False,
              'manual_seed': 1994,
#               "learning_rate": 1e-5,
              #'weight_decay': 0.01,
              "save_eval_checkpoints": False,
              "save_model_every_epoch": False,
              'no_cache':True,
              "silent": False,
              "use_early_stopping": True,
              #"early_stopping_delta": 0.015,
#               "early_stopping_metric": "mcc",
#               "early_stopping_metric_minimize": False,
#               "early_stopping_patience": 3,
#               "evaluate_during_training_steps": 1000,
              "reprocess_input_data": True
              }

In [17]:
def get_model():
    model = MultiLabelClassificationModel("roberta", "flax-sentence-embeddings/all_datasets_v3_roberta-large", use_cuda=True, num_labels=12, args=model_args)                            
    return model

In [18]:
from transformers import logging
logging.set_verbosity_warning()

In [19]:
sub = pd.DataFrame()

err=[]
y_pred_tot=[]
i=1

fold=KFold(n_splits=10, shuffle=True, random_state=1994)

for train_index, test_index in fold.split(train, train['labels'].tolist()):
    train1_trn, train1_val = train.iloc[train_index], train.iloc[test_index]
    model = get_model()
    gc.collect()

    model.train_model(train1_trn)
    score, raw_outputs_val, wrong_preds = model.eval_model(train1_val) 
#     raw_outputs_val = softmax(raw_outputs_val, axis=1)
    #raw_outputs_val = np.clip(raw_outputs_val,0.05,0.95)
    print('Log_Loss:', log_loss(train1_val['labels'].tolist(), raw_outputs_val))
    err.append(log_loss(train1_val['labels'].tolist(), raw_outputs_val))
    predictions, raw_outputs_test = model.predict(test['text'].tolist())
#     raw_outputs_test = softmax(raw_outputs_test, axis=1) 
    y_pred_tot.append(raw_outputs_test)

print(np.mean(err, 0))

y_pred = np.mean(y_pred_tot, 0)

Downloading:   0%|          | 0.00/650 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at flax-sentence-embeddings/all_datasets_v3_roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Running Evaluation:   0%|          | 0/77 [00:00<?, ?it/s]

Log_Loss: 3.050347885284166


  0%|          | 0/2631 [00:00<?, ?it/s]

  0%|          | 0/329 [00:00<?, ?it/s]

3.0297134921643254


In [20]:
test = pd.read_csv("/kaggle/input/uhack-sentiments-20/test.csv")

In [21]:
test.iloc[:,2:] = y_pred

In [22]:
sub = test.iloc[:,2:]

In [23]:
sub.head()

,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0.010201,0.008353,0.002486,0.068538,0.007583,0.005582,0.007779,0.815449,0.005187,0.990566,0.005122,0.003577
1,0.000663,0.000331,0.001073,0.001139,0.000898,0.998539,0.000701,0.000688,0.001419,0.003052,0.000985,0.999056
2,0.001358,0.000534,0.001031,0.002049,0.001971,0.003479,0.004290,0.000738,0.001618,0.003535,0.996085,0.997798
3,0.000819,0.000315,0.000752,0.001306,0.000978,0.998228,0.000710,0.000541,0.000930,0.001917,0.001665,0.999332
4,0.000591,0.000351,0.001233,0.000805,0.000900,0.001361,0.001340,0.000357,0.001290,0.995823,0.001851,0.996682


In [24]:
sub.to_csv("v25.csv", index=False)